In [1]:
!pip install --upgrade google-auth google-auth-oauthlib

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [ ]:
!.\python_venv\Scripts\activate.bat

In [1]:
HTTP GET /accounts/google/login/callback/?state=nI3vJ9c3AXSl&code=4/0AWtgzh48E6IztbDHlNJuPhTCj7ZWpbpdkesXsKBql2pWJi7x4Pt1ztlOVKTYMNdPX9v-Kg
&scope=email%20profile%20openid%20https://www.googleapis.com/auth/userinfo.email%20https://www.googleapis.com/auth/userinfo.profile&authuser=1&hd=nyu.edu
&prompt=none 302 [0.27, 127.0.0.1:52716]

HTTP GET /accounts/google/login/callback/?state=jN1V5m8FFZqk5LWQAB9snUkXEOHKvh&code=4%2F0AWtgzh7GfMXTwaynNBPXBCfV93Jy4dNlf0t4X2MlalpbIpxK4WiXO-1nbNxH_jmHq7dyYw
&scope=email+profile+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.profile+openid
&authuser=2
&prompt=consent 200 [0.25, 127.0.0.1:52825]


In [2]:
import os
from django.test import RequestFactory
from django.middleware import csrf
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "soulspark_backend.settings")
os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'
os.environ['CSRF_COOKIE_SECURE'] = 'False'

In [3]:
# Create a request object using RequestFactory
factory = RequestFactory()
url = '/fetch_chat_history/?lines=5&bot_id=1'
request = factory.get(url)

In [4]:
csrf_token = csrf.get_token(request)
csrf_token

'O4RIVfvwebbsjE79q8Pd1cy0xPiOwiiGY58IzWAiw4tSmu0v3FjRQUKad9uDUz6r'

In [5]:
import google.auth
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import Flow

# Create a new OAuth2 flow object
flow = Flow.from_client_secrets_file(
    './client_secret.json',
    scopes=['email','profile','openid'],
    redirect_uri="http://localhost:8000/accounts/google/login/callback/"
)


# Generate the authorization URL and prompt the user to authorize the app
authorization_url, state = flow.authorization_url(prompt='consent',
                                                  include_granted_scopes='true',
#                                                   state=csrf_token  # include the CSRF token as the state parameter
)

# Print the authorization URL and prompt the user to visit it in their web browser
print('Please visit this URL to authorize the application:', authorization_url)

# Once the user has authorized the app, prompt them to enter the authorization code
authorization_code = input('Enter the authorization code: ')

Please visit this URL to authorize the application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=485503899387-03u1pvv94g1k01tf9rhv7nno51tbfmls.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8000%2Faccounts%2Fgoogle%2Flogin%2Fcallback%2F&scope=email+profile+openid&state=jN1V5m8FFZqk5LWQAB9snUkXEOHKvh&prompt=consent&include_granted_scopes=true&access_type=offline
Enter the authorization code: 4%2F0AWtgzh7GfMXTwaynNBPXBCfV93Jy4dNlf0t4X2MlalpbIpxK4WiXO-1nbNxH_jmHq7dyYw


In [6]:
# Exchange the authorization code for an access token
flow.fetch_token(code=authorization_code)

# Get the access token from the OAuth2 flow object
credentials = flow.credentials
access_token = credentials.token
print('Access token:', access_token)

InvalidGrantError: (invalid_grant) Malformed auth code.

In [ ]:
http://localhost:8000/accounts/google/login/callback/?state=kLx1v6HO5hGh7DYLLVNUubZaFfMQTPt15wHUqZWYAgrbFj6COOwHe08i9myGFsoe&code=4%2F0AWtgzh6n7iSn7musBMduRqWr4N9AT_aDc8Lvij4bmjtJE24BRPcckbpJ2wilNhpeFHcg6w&scope=email+profile+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.profile&authuser=2&prompt=consent

In [ ]:
import requests

url = "http://localhost:8000/api/fetch_chat_history/"
headers = {
    "Authorization": "Bearer <your_access_token>",
    "Content-Type": "application/json"
}
params = {
    "lines": 5,
    "bot_id": 1
}

response = requests.post(url, headers=headers, json=params)
print(response.json())